In [20]:
import sys
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

sys.path.append('../../../batchflow')

from batchflow import B, V, F, R, P, C, Dataset, Batch, action, inbatch_parallel, ImagesBatch, Pipeline
from batchflow.models.metrics import ClassificationMetrics
from batchflow.models.torch.layers import Dense
from batchflow.models.torch import TorchModel
from batchflow.opensets import MNIST
from batchflow import best_practice

In [21]:
BATCH_SIZE = 64
mnist = MNIST()

DownloadingDownloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
 http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
Extracting /var/folders/d0/4xbr3ghj1nz4yr0d97gg0wkc0000gn/T/train-labels-idx1-ubyte.gz
http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gzExtracting 
Extracting /var/folders/d0/4xbr3ghj1nz4yr0d97gg0wkc0000gn/T/t10k-images-idx3-ubyte.gz
/var/folders/d0/4xbr3ghj1nz4yr0d97gg0wkc0000gn/T/train-images-idx3-ubyte.gz
Extracting /var/folders/d0/4xbr3ghj1nz4yr0d97gg0wkc0000gn/T/t10k-labels-idx1-ubyte.gz


In [72]:
class MyModel(TorchModel):
    @classmethod
    def default_config(cls):
        config = TorchModel.default_config()
        config['initial_block/layout'] = 'f'
        config['initial_block/units'] = 10
        return config

    def build_config(self, names=None):
        config = super().build_config(names)
        config['head/units'] = self.num_classes('targets')
        return config

    @classmethod
    def body(cls, **kwargs):
        kwargs = cls.get_defaults('body', kwargs)
        return Dense(**kwargs)

In [73]:
model_config = {
    'inputs': dict(images={'shape': (28, 28, 1)},
                   labels={'classes': 10}),
    'loss': 'ce',
    'optimizer': 'Adam',
    'initial_block/inputs': 'images',
    'head/units': 10,
    'output': dict(predicted=['proba', 'labels'])
}

In [74]:
train_pipeline = (mnist.train.p
    .init_variable('loss_history', init_on_each_run=list)
    .init_model('dynamic', MyModel, 'my_model', model_config)
    .train_model('my_model', B('images'), B('labels'),
                 fetches='loss', save_to=V('loss_history'), mode='a'))

In [75]:
train_pipeline.run(BATCH_SIZE, shuffle=True, n_epochs=1, bar=True)















  0%|          | 0/938 [00:00<?, ?it/s]

TypeError: new() received an invalid combination of arguments - got (NoneType, numpy.int64), but expected one of:
 * (torch.device device)
 * (torch.Storage storage)
 * (Tensor other)
 * (tuple of ints size, torch.device device)
      didn't match because some of the arguments have invalid types: ([31;1mNoneType[0m, [31;1mnumpy.int64[0m)
 * (object data, torch.device device)
      didn't match because some of the arguments have invalid types: ([31;1mNoneType[0m, [31;1mnumpy.int64[0m)
